In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from pymystem3 import Mystem
import string
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm_notebook as tqdm

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.12.0


### Работа с данными (1 балл)

Загрузите датасет, с которым вы работали во время соревнования на kaggle. Преобразуйте его в формат, удобный для обучения модели. В качетсве фичей используйте эмбединги слов.

In [2]:
# Your code here
from sklearn.utils import resample
data = pd.read_csv('train.csv', index_col= False)
predict = pd.read_csv('test.csv', index_col= False)

In [3]:
data.head()

,id,text,label
0,0,ж 44 позн с мужч от 40 лет 0678935919,1
1,1,уже и не вспомню где это увидел,2
2,2,", sep=';', decimal=',' может изза этого",2
3,3,НУЖНА ДЕВ ДЛЯ С\\О -19л.0637048428смс,1
4,4,.М54.ИЩУ Д.Ж. С-П.ДЛЯ ЛАСК...НЕКМ.0508665030....,1


In [4]:
Y = np.array(data.label)
X = np.array(data.text)
x_test = np.array(predict.text)


x_train_variable = X[0:100:]
y_train = Y[0:100:]
x_test_variable = X[100:200:]
y_test =Y[100:200:]

In [5]:
def get_top_n_words(corpus, n=None):

    vec = CountVectorizer(tokenizer=lemmatize_with_mystem).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    top = words_freq[:n]
    top, _ = zip(*top)
    
    return set(top)

#get_top_n_words(words, 500)

In [6]:
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2
sentence_size = 15
vocab_size = 10
model_dir = tempfile.mkdtemp()
mystem = Mystem()

tr = str.maketrans(string.punctuation, ' '*len(string.punctuation))
gg =x_train_variable[0:6:]

def lemmatize_with_mystem(text):
    return [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace()]

def tokens(arra):
    result = []
    token = []
    for element in arra:
        element = element.lower()
        element = element.translate(tr)
        element = lemmatize_with_mystem(element)
        result.append(element)        
    return result
        
def to_numbers(arra):
    #punct elimanation
    for i in tqdm(range(arra.size)):
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
        arra[i] = arra[i].translate(translator)
    #vectorize
    vectorizer = CountVectorizer(tokenizer=lemmatize_with_mystem)
    vectorizer.fit_transform(arra).todense
    
    my_vocab = get_top_n_words(arra, vocab_size)
    #making offset
    for url in vectorizer.vocabulary_:
        vectorizer.vocabulary_[url] += index_offset
    tokenized = tokens(arra)
    #map with numbers
    print(tokenized)
    for i in tqdm(range(len(tokenized))):
        #start_id
        tokenized[i] = [start_id] + tokenized[i]
        for j  in range(1, len(tokenized[i])):
            if tokenized[i][j] in my_vocab:
                tokenized[i][j] = vectorizer.vocabulary_[tokenized[i][j]]
            else:
                tokenized[i][j] = oov_id
    print(tokenized)               
    return tokenized  
#gg = to_numbers(gg)
x_train_variable = to_numbers(x_train_variable)
x_test_variable = to_numbers(x_test_variable)
# print("done")



[['ж', '44', 'позн', 'с', 'мужч', 'от', '40', 'год', '0678935919'], ['уже', 'и', 'не', 'вспомнить', 'где', 'это', 'увидеть'], ['sep', 'decimal', 'может', 'изза', 'это'], ['нужный', 'дева', 'для', 'с', 'о', '19л', '0637048428смс'], ['м54', 'искать', 'д', 'ж', 'с', 'п', 'для', 'ласка', 'некм', '0508665030', 'смс', 'задерж', 'ждать'], ['я', 'говорить', 'от', 'они', 'спать', 'постоянно', 'хотеться'], ['познакомиться', 'с', 'адекватный', 'мужчина', 'с', 'о', '0668847450'], ['модер', 'доченька', 'звать', 'арина', 'папа', 'дима', 'мама', 'настя', 'малышка', 'рождаться', '14', '05', '13', 'задержы'], ['но', 'это', 'не', 'попс', 'не', 'надо'], ['пар', 'пообщий', 'с', 'дева', 'по', 'смс', '0935695491'], ['не', 'смотреть', 'на', 'паблик', 'или', 'смотреть', 'как', 'на', 'один', 'из', 'прочий', 'фолд'], ['дно', 'дзержинск', 'пар', 'искать', 'девушка', 'милый', 'для', 'сера', 'отноша', '093', '57', '43', '901'], ['п', 'п', 'до', '26', 'сейчас', 'днепр'], ['да', 'надо', 'растягивать', 'фон', 'на', 


[[1, 2, 2, 2, 514, 2, 2, 2, 2, 2], [1, 2, 282, 383, 2, 2, 628, 2], [1, 2, 2, 2, 2, 628], [1, 2, 2, 234, 514, 2, 2, 2], [1, 2, 295, 2, 2, 514, 2, 234, 2, 2, 2, 2, 2, 2], [1, 630, 2, 2, 2, 2, 2, 2], [1, 2, 514, 2, 2, 514, 2, 2], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 2, 628, 383, 2, 383, 2], [1, 2, 2, 514, 2, 2, 2, 2], [1, 383, 2, 374, 2, 2, 2, 2, 374, 2, 2, 2, 2], [1, 2, 2, 2, 295, 222, 2, 234, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 374, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 514, 2, 2], [1, 2, 2, 630, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 383, 2, 2, 2, 2, 164, 2, 2], [1, 2, 2, 2, 2, 164, 2, 2], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 374, 2, 282, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 514, 2, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 2, 222, 295, 2, 2, 282, 2, 234, 2, 630, 2, 2, 2], [1, 2, 2, 282, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 514, 2, 2, 2, 282, 514, 2, 2, 2, 2, 2], 


[['у', 'я', 'хороший', 'приват', 'xgboost', 'без', 'блендинг', 'как', 'так', 'то', '😁'], ['в', 'кредитный', 'скорринг', 'где', 'дисбаланс', 'сильный', 'тип', '1процента', 'дефолт', 'использовать', 'аук'], ['я', 'не', 'попадать', 'из', 'топ', '100', 'в', 'топ', '600'], ['девушка', 'хотеть', 'без', 'обязательство', 'быть', 'где'], ['я', 'на', 'корпоратив', 'ну', 'сам', 'понимать', 'я', 'не', 'контролировать', 'себя', 'даже', 'знак', 'препинание', 'не', 'верно', 'расставлять'], ['я', 'казаться', 'что', 'у', 'человек', 'каша', 'в', 'голова', 'и', 'он', 'надо', 'сначала', 'программировать', 'научаться'], ['м', 'искать', 'д', 'ж', 'сие', 'пара', 'для', 'весь', '068', '887', '92', '59', 'смс', 'зад', 'пожа'], ['познакомиться', 'с', 'девушка', 'из', 'запорожье', 'парень', '26', 'год', 'тело', '0930778529'], ['ну', 'я', 'бы', '25', 'единоразовый', 'мочь', 'и', 'платить', 'но', 'они', 'же', 'каждый', 'месяц', 'требовать'], ['два', 'девушка', 'скучать', 'парень', 'до', '22', 'звонить', 'писать',


[[1, 2, 670, 2, 2, 2, 2, 2, 2, 2, 609, 2], [1, 179, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 670, 387, 2, 2, 2, 2, 179, 2, 2], [1, 232, 2, 2, 2, 2, 2], [1, 670, 380, 2, 2, 2, 2, 670, 387, 2, 2, 2, 2, 2, 387, 2, 2], [1, 670, 2, 657, 2, 2, 2, 179, 2, 291, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 241, 2, 2, 2, 2, 2, 2, 2, 2], [1, 2, 534, 232, 2, 2, 2, 2, 2, 2, 2], [1, 2, 670, 2, 2, 2, 2, 291, 2, 2, 2, 2, 2, 2, 2], [1, 2, 232, 2, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 670, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 670, 291, 2, 2, 2, 2, 2, 2], [1, 2, 2, 232, 2, 2, 2, 2], [1, 2, 2, 609, 670, 2, 380, 2, 2, 670, 380, 2, 2, 2], [1, 534, 2, 2, 380, 2, 2, 2, 387, 2, 2, 2, 2, 2, 2, 2, 2, 2, 609, 2], [1, 2, 2, 387, 2, 2, 2, 2, 2, 2, 2, 2, 291, 2, 2, 387, 2], [1, 2, 2, 291, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 241, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 2, 2, 609, 2, 2, 179, 2, 291, 2, 179, 2], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 534, 2, 2], [1, 2, 2, 2, 2, 2, 2], [1, 2, 2, 2, 657, 2, 2], [1

In [7]:
print(len(y_train), "train sequences")
print(len(y_test), "test sequences")

print("Pad sequences (samples x time)")
x_train = sequence.pad_sequences(x_train_variable, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test_variable, 
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post', 
                                value=pad_id)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

100 train sequences
100 test sequences
Pad sequences (samples x time)
x_train shape: (100, 15)
x_test shape: (100, 15)


In [8]:
# проверим на рандоме
# x_train = np.random.randint(1, 10, size=(100,10))
# y_train = np.random.randint(0, 1, size=(100,1))
# x_test = np.random.randint(1, 10, size=(100,10))
# y_test = np.random.randint(0, 1, size=(100,1))
# x_len_train = np.random.randint(1, 10, size=(100,1))
# x_len_test = np.random.randint(1, 10, size=(100,1))

In [9]:
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    # Подача данных для обучения
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    # Подача тестовых данных
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [10]:
column = tf.feature_column.categorical_column_with_identity('x', vocab_size)
classifier = tf.estimator.LinearClassifier(feature_columns=[column], model_dir=os.path.join(model_dir, 'bow_sparse'),n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpe5wy72cr/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13297b898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
all_classifiers = {}
def train_and_evaluate(classifier):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=5000)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [12]:
# classifier.predict?

In [13]:
train_and_evaluate(classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpe5wy72cr/bow_sparse/model.ckpt.


InvalidArgumentError: assertion failed: [1 2 2...] [10]
	 [[node linear/linear_model/linear_model/x/assert_less_than_num_buckets/Assert/AssertGuard/Assert (defined at <ipython-input-11-92da1b3531b8>:5)  = Assert[T=[DT_INT64, DT_INT64], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](linear/linear_model/linear_model/x/assert_less_than_num_buckets/Assert/AssertGuard/Assert/Switch, linear/linear_model/linear_model/x/assert_less_than_num_buckets/Assert/AssertGuard/Assert/Switch_1, linear/linear_model/linear_model/x/assert_less_than_num_buckets/Assert/AssertGuard/Assert/Switch_2)]]

Caused by op 'linear/linear_model/linear_model/x/assert_less_than_num_buckets/Assert/AssertGuard/Assert', defined at:
  File "/Users/bulat/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/bulat/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/bulat/anaconda3/lib/python3.6/site-packa

In [233]:
embedding_size = 10
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
classifier = tf.estimator.DNNClassifier(
    hidden_units=[100],
    feature_columns=[word_embedding_column], 
    model_dir=os.path.join(model_dir, 'bow_embeddings'))
train_and_evaluate(classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zw/2w6s_cjx4h9dj9fjz5t8tr800000gn/T/tmpo4y5uwuf/bow_embeddings', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13af259b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorf

Exception ignored in: <generator object Estimator.predict at 0x13fad80f8>
Traceback (most recent call last):
  File "/Users/bulat/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/estimator.py", line 605, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/Users/bulat/anaconda3/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/Users/bulat/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 5229, in get_controller
    yield g
  File "/Users/bulat/anaconda3/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/Users/bulat/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 5045, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:tensorflow:loss = 73.24349, step = 1
INFO:tensorflow:global_step/sec: 470.699
INFO:tensorflow:loss = 0.021560593, step = 101 (0.214 sec)
INFO:tensorflow:global_step/sec: 831.629
INFO:tensorflow:loss = 0.008867321, step = 201 (0.120 sec)
INFO:tensorflow:global_step/sec: 895.824
INFO:tensorflow:loss = 0.005340683, step = 301 (0.112 sec)
INFO:tensorflow:global_step/sec: 590.615
INFO:tensorflow:loss = 0.003744308, step = 401 (0.171 sec)
INFO:tensorflow:global_step/sec: 395.307
INFO:tensorflow:loss = 0.0028494415, step = 501 (0.252 sec)
INFO:tensorflow:global_step/sec: 517.494
INFO:tensorflow:loss = 0.0022826989, step = 601 (0.193 sec)
INFO:tensorflow:global_step/sec: 798.799
INFO:tensorflow:loss = 0.0018941434, step = 701 (0.126 sec)
INFO:tensorflow:global_step/sec: 728.948
INFO:tensorflow:loss = 0.0016124828, step = 801 (0.137 sec)
INFO:tensorflow:global_step/sec: 839.996
INFO:tensorflow:loss = 0.0013996754, step = 901 (0.119 sec)
INFO:tensorflow:global_step/sec: 849.386
INFO:tensorf

AssertionError: Do not use tf.reset_default_graph() to clear nested graphs. If you need a cleared graph, exit the nesting and create a new graph.

### CNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- Dense

Параметры модели подберите сами.

In [152]:
head = tf.contrib.estimator.binary_classification_head()

def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training)

    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        

    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
  
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        params=params)

/home/agavrilenko/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2rnvsyug/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb23406b518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [153]:
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp2rnvsyug/cnn/model.ckpt.


InvalidArgumentError: indices[0,1] = 59192 is not in [0, 500)
	 [[node EmbedSequence/embedding_lookup (defined at /home/agavrilenko/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/layers/python/layers/encoders.py:142)  = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@head/Adam/Assign_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](EmbedSequence/embeddings/read, IteratorGetNext:1, EmbedSequence/embedding_lookup/axis)]]

Caused by op 'EmbedSequence/embedding_lookup', defined at:
  File "/home/agavrilenko/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/agavrilenko/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/agavrilenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/agavrilenko/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py",

### RNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- RNN
- Dense

Параметры модели подберите сами.

In [ ]:
# your code here

### LSTM model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- LSTM
- Dense

Параметры модели подберите сами.

In [ ]:
# your code here

Сравните все три реализованные модели по времени и по качеству классификации. Какая лучше? Как думаете, почему?

<b>Вывод:</b>

### Baseline model (2 балла)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Параметры для Conv1D, MaxPooling1D и RNN подберите сами.

In [ ]:
# your code here

### Dropout (1 балл)

Добавьте dropout к baseline моделе. Пример модефицированной модели:
- Dropout
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Подберите параметры.

In [ ]:
# your code here

Улучшило ли результат классификации использование Dropout? Как думаете, почему?

<b>Вывод:</b>

### Bidirectional (1 балл)

Вместо RNN в первой модели используйте biRNN. Пример модефецированной модели:
- Conv1D, activation='relu'
- MaxPooling1D
- biRNN
- Dense

Подберите параметры.

In [ ]:
# your code here

Дало ли буст использовании biRNN вместо RNN? Сильный? Как думаете, почему?

<b>Вывод:</b>

### Custom model (2 балла)

Подберите архитектуру сети, используя сверточные слои и слои макспулинга и превзойдите качество полносвязной сети.

Подберите архитектуру сети, используя следующие типы слоев:
- Conv1D
- MaxPooling1D
- RNN
- LSTM
- GRU
- Dropout
- Dense

Можно использовать любые из описанных слоев в любом количестве и в любом порядке. Не обязательно использовать все описанные слои. 

Настройте параметры. Превзойдите лучшее качество, полученное раннее. 

In [ ]:
# your code here